In [12]:
import gym
import tensorflow as tf
from keras.optimizers import Adam
import numpy as np
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [13]:
env = gym.make("CartPole-v0")

states = env.observation_space.shape[0]
actions = env.action_space.n

In [14]:
actions,states

(2, 4)

In [15]:
def create_model(states,action):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(1,states)),
        tf.keras.layers.Dense(24,activation="relu"),
        tf.keras.layers.Dense(24,activation="relu"),
        tf.keras.layers.Dense(actions)
    ])

    return model

In [16]:
def build_agent(model,actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=30000,window_length=1)
    dqn = DQNAgent(model=model,memory = memory,policy=policy,
                   nb_actions=actions,nb_steps_warmup=10,target_model_update=1e-2)
    return dqn

In [17]:
model = create_model(states,actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [18]:
dqn = build_agent(model,actions)
dqn.compile(Adam(lr=1e-3),metrics=['mae'])
dqn.fit(env, nb_steps=50000)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 91s 9ms/step - reward: 1.0000
122 episodes - episode_reward: 81.459 [9.000, 200.000] - loss: 2.855 - mae: 18.488 - mean_q: 37.546

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 89s 9ms/step - reward: 1.0000
53 episodes - episode_reward: 187.679 [154.000, 200.000] - loss: 7.411 - mae: 39.159 - mean_q: 79.201

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 92s 9ms/step - reward: 1.0000
52 episodes - episode_reward: 191.615 [146.000, 200.000] - loss: 8.157 - mae: 43.544 - mean_q: 87.806

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 92s 9ms/step - reward: 1.0000
53 episodes - episode_reward: 190.830 [114.000, 200.000] - loss: 6.403 - mae: 42.966 - mean_q: 86.195

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 89s 9ms/step - reward: 1.0000: 0s -

In [19]:
scores = dqn.test(env, nb_episodes=10,visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
200.0


In [20]:
dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200


In [ ]:
dqn.save_weights("training_weights/dqn_weights")

In [21]:
env.close()